# Часть 2. Соединение и предобработка скачанных отзывов 

In [3]:
from sklearn import tree
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import SVG
#from graphviz import Source
from IPython.display import display

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor

In [4]:
#увеличиваю отображение таблиц в ячейках вывода, чтобы показывало все данные
#pd.options.display.max_rows = 2250
#pd.options.display.max_columns = 100

## 2.1 подгружаю ранее скачанные отзывы с площадок

In [57]:
df_salyt_g= pd.read_csv('D:/Документы/Python/Площадки/google_review_салют.csv' )
df_salyt_g["location"] = 'Парк Горького. Салют'

In [58]:
df_piramidy_g= pd.read_csv('D:/Документы/Python/Площадки/google_review_piramidy2.csv' )
df_piramidy_g["location"] = 'Бунинские луга. Пирамиды'

In [59]:
df_salyt_yt= pd.read_csv('D:/Документы/Python/jupyter/df_salyt_youtube.csv' )#предобработанные

In [60]:
df_s_p_ya= pd.read_csv('df_salyt_piramidy_ya.csv' )#предобработанные

## Объединю отзывы из google карт

In [61]:
df = pd.concat([df_salyt_g, df_piramidy_g],ignore_index=True, axis=0).rename(columns={'rating':'date','duration':'comment'})
df.head(20)

,name,date,review,comment,location
0,Дмитрий,9 месяцев назад,5 звезд,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют
1,Михаил Лось,11 месяцев назад,5 звезд,Здесь очень весело!),Парк Горького. Салют
2,Vika Andreeva,год назад,5 звезд,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют
3,Светлана Рабинович,год назад,5 звезд,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют
4,Игорь Тухкин,год назад,5 звезд,NaN,Парк Горького. Салют
5,Андрей Харитонов,год назад,4 звезды,NaN,Парк Горького. Салют
6,Денис Пупыкин,год назад,5 звезд,Отличное место для игр с детьми))),Парк Горького. Салют
7,Ярослав Кокосов (ilu),год назад,3 звезды,NaN,Парк Горького. Салют
8,Серега Крапотухин,год назад,5 звезд,NaN,Парк Горького. Салют
9,Эдуард Зуфаров,год назад,5 звезд,Отличная плошядка для прогулок с детьми,Парк Горького. Салют


## Преобразования таблицы

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      1052 non-null   object
 1   date      1052 non-null   object
 2   review    1052 non-null   object
 3   comment   369 non-null    object
 4   location  1052 non-null   object
dtypes: object(5)
memory usage: 41.2+ KB


In [62]:
df['review'] = df['review'].str.split().str[0]
df = df.astype({'review': np.int8})
df.head(20)

,name,date,review,comment,location
0,Дмитрий,9 месяцев назад,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют
1,Михаил Лось,11 месяцев назад,5,Здесь очень весело!),Парк Горького. Салют
2,Vika Andreeva,год назад,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют
3,Светлана Рабинович,год назад,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют
4,Игорь Тухкин,год назад,5,NaN,Парк Горького. Салют
5,Андрей Харитонов,год назад,4,NaN,Парк Горького. Салют
6,Денис Пупыкин,год назад,5,Отличное место для игр с детьми))),Парк Горького. Салют
7,Ярослав Кокосов (ilu),год назад,3,NaN,Парк Горького. Салют
8,Серега Крапотухин,год назад,5,NaN,Парк Горького. Салют
9,Эдуард Зуфаров,год назад,5,Отличная плошядка для прогулок с детьми,Парк Горького. Салют


In [29]:
df['date'].value_counts().reset_index(name='Amount')

,index,Amount
0,3 года назад,363
1,год назад,263
2,2 года назад,229
3,4 года назад,185
4,10 месяцев назад,5
5,5 лет назад,4
6,9 месяцев назад,2
7,11 месяцев назад,1


In [63]:
df = df.replace({'9 месяцев назад': '2022', '11 месяцев назад': '2022', '10 месяцев назад': '2022', 'год назад': '2021', '3 года назад': '2019', '2 года назад':'2020', '4 года назад':'2018', '5 лет назад':'2017'})

In [64]:
df["resource"] = 'google'


In [65]:
df_all = pd.concat([df,df_salyt_yt,df_s_p_ya ],ignore_index=True, axis=0)

In [66]:
df_all = df_all.drop(columns = ['Unnamed: 0'],axis = 1)

In [67]:
df2 = df_all.copy ()


In [68]:
df2.head(20)

,name,date,review,comment,location,resource
0,Дмитрий,2022,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют,google
1,Михаил Лось,2022,5,Здесь очень весело!),Парк Горького. Салют,google
2,Vika Andreeva,2021,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют,google
3,Светлана Рабинович,2021,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют,google
4,Игорь Тухкин,2021,5,NaN,Парк Горького. Салют,google
5,Андрей Харитонов,2021,4,NaN,Парк Горького. Салют,google
6,Денис Пупыкин,2021,5,Отличное место для игр с детьми))),Парк Горького. Салют,google
7,Ярослав Кокосов (ilu),2021,3,NaN,Парк Горького. Салют,google
8,Серега Крапотухин,2021,5,NaN,Парк Горького. Салют,google
9,Эдуард Зуфаров,2021,5,Отличная плошядка для прогулок с детьми,Парк Горького. Салют,google


## 2.2 Определю и разделю имена на мужчин и женщин

####  c помощью генератора  имен (https://www.fakenamegenerator.com/order.php) создам базу данных с которой буду сравнивать имена в датасете

In [69]:
fake_name_ru= pd.read_csv('D:/Документы/Python/Площадки/FakeNameGenerator.com_09efa2a5.csv' )
fake_name_en= pd.read_csv('D:/Документы/Python/Площадки/FakeNameGenerator.com_3b24f591.csv' )
fake_name_ru_en= pd.read_csv('D:/Документы/Python/Площадки/FakeNameGenerator.com_nnk4dfb59c5.csv')
fake_name_add= pd.read_csv('D:/Документы/Python/Площадки/FakeNameGenerator.com_add.csv' )
fake_name_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100016 entries, 0 to 100015
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Gender     100016 non-null  object
 1   GivenName  100016 non-null  object
 2   Surname    100016 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [70]:
fake_names = pd.concat([fake_name_ru, fake_name_en, fake_name_ru_en,fake_name_add],ignore_index=True, axis=0)

In [71]:
male_names = set(fake_names[fake_names['Gender'] == 'male']['GivenName'].str.lower())
male_surnames = set(s[-3:] for s in fake_names[fake_names['Gender'] == 'male']['Surname'])

female_names= set(fake_names[fake_names['Gender'] == 'female']['GivenName'].str.lower())
female_surnames = set(s[-3:] for s in fake_names[fake_names['Gender'] == 'female']['Surname'])


In [72]:
def how_you(name):
    score = 0
    try:    
        if name.lower().strip() in female_names:
            score += 2

        if name.lower().strip() in male_names:
            score -= 2

    
        if name[-3:].lower().strip() in female_surnames:
            score += 1

        if name[-3:].lower().strip() in male_surnames:
            score -= 1   
    
    except :
        pass  
    return score   
 

In [73]:
df3=df2.copy()

In [74]:
df3['name'] = df2['name'].str.split(' ').str[0]
df3['surname'] = df2['name'].str.split(' ').str[1]
df3.head(20)

,name,date,review,comment,location,resource,surname
0,Дмитрий,2022,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют,google,NaN
1,Михаил,2022,5,Здесь очень весело!),Парк Горького. Салют,google,Лось
2,Vika,2021,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют,google,Andreeva
3,Светлана,2021,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют,google,Рабинович
4,Игорь,2021,5,NaN,Парк Горького. Салют,google,Тухкин
5,Андрей,2021,4,NaN,Парк Горького. Салют,google,Харитонов
6,Денис,2021,5,Отличное место для игр с детьми))),Парк Горького. Салют,google,Пупыкин
7,Ярослав,2021,3,NaN,Парк Горького. Салют,google,Кокосов
8,Серега,2021,5,NaN,Парк Горького. Салют,google,Крапотухин
9,Эдуард,2021,5,Отличная плошядка для прогулок с детьми,Парк Горького. Салют,google,Зуфаров


### так как встречаются имена и фамилии не на своих местах, буду применять функцию по отдельности на каждую колонку

In [75]:
df3['gender'] = df3.apply(lambda x: how_you(x['name']), axis =  1)
df3.head(20)

,name,date,review,comment,location,resource,surname,gender
0,Дмитрий,2022,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют,google,NaN,-2
1,Михаил,2022,5,Здесь очень весело!),Парк Горького. Салют,google,Лось,-2
2,Vika,2021,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют,google,Andreeva,3
3,Светлана,2021,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют,google,Рабинович,2
4,Игорь,2021,5,NaN,Парк Горького. Салют,google,Тухкин,-2
5,Андрей,2021,4,NaN,Парк Горького. Салют,google,Харитонов,-2
6,Денис,2021,5,Отличное место для игр с детьми))),Парк Горького. Салют,google,Пупыкин,-2
7,Ярослав,2021,3,NaN,Парк Горького. Салют,google,Кокосов,-2
8,Серега,2021,5,NaN,Парк Горького. Салют,google,Крапотухин,-2
9,Эдуард,2021,5,Отличная плошядка для прогулок с детьми,Парк Горького. Салют,google,Зуфаров,-2


In [43]:
df3['gender'].value_counts().reset_index(name='Amount')

,index,Amount
0,-2,765
1,2,584
2,0,422
3,3,100
4,1,34
5,-1,29
6,-3,17


In [76]:
df3[df3['gender'] == 0].head(10)

,name,date,review,comment,location,resource,surname,gender
11,Pibody,2021,5,Площадка просто бомба! Но желающих одновременн...,Парк Горького. Салют,google,Nina,0
13,Милый,2021,5,NaN,Парк Горького. Салют,google,Кролик,0
16,x1zw1,2021,5,NaN,Парк Горького. Салют,google,NaN,0
17,Анжелика,2021,5,NaN,Парк Горького. Салют,google,Ягодина,0
23,Paul,2021,4,(Переведено Google) Невероятная бесплатная игр...,Парк Горького. Салют,google,Bailey,0
24,Светланчик,2021,5,Даже взрослым детям интересно,Парк Горького. Салют,google,Шарун,0
34,Vika_the_,2021,5,"Очень интересное место, дети в восторге",Парк Горького. Салют,google,Likee,0
41,Саша,2021,5,NaN,Парк Горького. Салют,google,Иванова,0
61,тузик,2021,5,Очень хорошая детская площадка. Есть зоны для ...,Парк Горького. Салют,google,007,0
62,Kudukhova,2021,5,NaN,Парк Горького. Салют,google,Ketevan,0


In [24]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951 entries, 0 to 1950
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      1951 non-null   object
 1   date      1951 non-null   object
 2   review    1205 non-null   object
 3   comment   1268 non-null   object
 4   location  1951 non-null   object
 5   resource  1951 non-null   object
 6   surname   1695 non-null   object
 7   gender    1951 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 122.1+ KB


In [77]:
df4 = df3.copy()

In [78]:
df4['gender2'] =df4.apply(lambda x: how_you(x['surname']), axis =  1)

In [79]:
df4['gender'] = df4[['gender','gender2']].sum(axis=1)

In [80]:
df4.head(10)

,name,date,review,comment,location,resource,surname,gender,gender2
0,Дмитрий,2022,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют,google,NaN,-2,0
1,Михаил,2022,5,Здесь очень весело!),Парк Горького. Салют,google,Лось,-2,0
2,Vika,2021,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют,google,Andreeva,3,0
3,Светлана,2021,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют,google,Рабинович,2,0
4,Игорь,2021,5,NaN,Парк Горького. Салют,google,Тухкин,-3,-1
5,Андрей,2021,4,NaN,Парк Горького. Салют,google,Харитонов,-2,0
6,Денис,2021,5,Отличное место для игр с детьми))),Парк Горького. Салют,google,Пупыкин,-3,-1
7,Ярослав,2021,3,NaN,Парк Горького. Салют,google,Кокосов,-3,-1
8,Серега,2021,5,NaN,Парк Горького. Салют,google,Крапотухин,-3,-1
9,Эдуард,2021,5,Отличная плошядка для прогулок с детьми,Парк Горького. Салют,google,Зуфаров,-3,-1


In [49]:
df4['gender'].value_counts().reset_index(name='Amount')

,index,Amount
0,2,471
1,-2,411
2,-3,384
3,0,335
4,3,190
5,-1,57
6,1,48
7,4,42
8,-4,13


In [81]:
df4 = df4.drop(columns = ['gender2'],axis = 1)

In [82]:
def converter_to_gender(x):
    
    if x>0:
        return 'female'
    if x<0:
        return 'male'
    if x ==0:
        return 'Nan'

In [83]:
df5 =df4.copy()

In [84]:
df5['gender'] =df5.apply(lambda x: converter_to_gender(x['gender']), axis =  1)

In [85]:
df5 = df5.reindex(columns=['name', 'surname', 'gender', 'date', 'review', 'comment', 'location', 'resource'])

In [86]:
df5['name'] = df2['name']

In [87]:
df5 = df5.drop(columns = ['surname'],axis = 1)
df5.head(20)

,name,gender,date,review,comment,location,resource
0,Дмитрий,male,2022,5,"Главное, ребёнку нравится. Но летом народу оче...",Парк Горького. Салют,google
1,Михаил Лось,male,2022,5,Здесь очень весело!),Парк Горького. Салют,google
2,Vika Andreeva,female,2021,5,Очень классное место.Очень весело.Особенно пон...,Парк Горького. Салют,google
3,Светлана Рабинович,female,2021,5,"Замечательный комплекс площадок, много качелей...",Парк Горького. Салют,google
4,Игорь Тухкин,male,2021,5,NaN,Парк Горького. Салют,google
5,Андрей Харитонов,male,2021,4,NaN,Парк Горького. Салют,google
6,Денис Пупыкин,male,2021,5,Отличное место для игр с детьми))),Парк Горького. Салют,google
7,Ярослав Кокосов (ilu),male,2021,3,NaN,Парк Горького. Салют,google
8,Серега Крапотухин,male,2021,5,NaN,Парк Горького. Салют,google
9,Эдуард Зуфаров,male,2021,5,Отличная плошядка для прогулок с детьми,Парк Горького. Салют,google


In [86]:
df5.to_csv('playground_reviews.csv', index=False)

In [82]:
df5.to_excel (r'playground_reviews.xlsx')

In [83]:
df5['gender'].value_counts().reset_index(name='Amount')  # остались ники которые невозможно распределить 

,index,Amount
0,male,865
1,female,751
2,Nan,335
